<a href="https://colab.research.google.com/github/fabiobrau/PCA/blob/main/notebooks/pca_on_real_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
## Load the data
import torch
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import numpy as np

data = MNIST(root='./data', download=True, train=False)

In [ ]:
## Center the data
X = np.array(data.data.float())/255
mean = X.mean()
print(f"Mean: {mean}")
X = (X - mean)
## Print the first 5 digits in row
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(X[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
plt.show()

##
N = X.shape[0]
n = X.shape[1]*X.shape[2]
X = X.reshape(N, -1)
print(f"N: {N}")
print(f"n: {n}")

In [ ]:
### Finding the principal directions with sklearn (three manners)
# Directly finding the eigen values of the X^TX
M = (X.T @ X)/N
eigenvalues, eigenvectors = np.linalg.eigh(M)
# sort in decreasing order
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]
#plot all the eigenvalues (x-axis in log scale)
plt.plot(eigenvalues)
plt.xscale('log')
plt.yscale('log')
# The rank of M never exceed n
plt.vlines(n,0, max(eigenvalues), colors="r")
plt.show()

In [178]:

## Second method: Using the singular value decomposition
U, S, V = np.linalg.svd(X, full_matrices=False)
V = V.T
print(f"Are the values the same? {np.allclose(eigenvalues, S**2/N, atol=1e-5)}")
print(f"Computational Error: {np.linalg.vector_norm((S**2)/N - eigenvalues)}")


Are the values the same? True
Computational Error: 9.966553079721052e-06


In [ ]:
### Showing the first 5 components
fig, ax = plt.subplots(1, 5)
fig.suptitle('First 5 eigen-digits')
fig.subplots_adjust(top=1.5)
for i in range(5):
    ax[i].imshow(V[:,i].reshape(28, 28), cmap='gray')
    ax[i].axis('off')
plt.show()

In [ ]:
## Computing the factors, and visualizing few of them

F = X@V
# plot the data grouped by clusters
fig, ax = plt.subplots(1,2,figsize=(10, 7.5))
for i in range(10):
    ax[0].scatter(F[data.targets == i, 0], F[data.targets == i, 1], label=f"Digit: {i}")
ax[0].set_title("First two factors computed with SVD")
ax[0].legend()
ax[0].set_xlabel('First factor')
ax[0].set_ylabel('Second Factor')
F_eigh = X@eigenvectors
for i in range(10):
    ax[1].scatter(F_eigh[data.targets == i, 0], F_eigh[data.targets == i, 1], label=f"Digit: {i}")
ax[1].set_title("First two factors computed with Eigh")
ax[1].legend()
ax[1].set_xlabel('First factor')
ax[1].set_ylabel('Second Factor')
plt.show()
print("Are they different?")

In [ ]:
## Visualization of the facotors with plotly in three dimensions
import plotly.express as px
import plotly.graph_objects as go

# set the colors form tab10 of matplot lib
colors = plt.cm.tab10(range(10))
# plot differnt digits with a differnt legend
px.scatter_3d(x=F[:, 0], y=F[:, 1], z=F[:, 2], color=data.targets)
#for i in range(10):
#  fig.add_scatter3d(connectgaps=None, x = F[data.targets == i,0], y = F[data.targets == i,1], z = F[data.targets == i,2],
#                 mode='markers',
#                 marker=dict(size=3, color="species"),
#                 name=f'Digit {i}',
#                 visible='legendonly'
#                 )
fig.show()


In [107]:
# Third method
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)
vals = (pca.singular_values_)
W = pca.components_.T
F = pca.transform(X)